In [2]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import csv
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import tqdm.notebook as tq
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transformers.logging.set_verbosity(transformers.logging.CRITICAL)

In [78]:
df = pd.read_csv("runs/02 EMNLP SAE-AAVE Pairs/aave_gen_txt.csv")
df.head()

,index,txt
0,0,[' to be his sister who']
1,1,[' earth!!\n\nthe']
2,2,"["" 4:15 I'm""]"
3,3,[' TO GET HIM TO DO']
4,4,['\nnot giving up all']


In [49]:
df.shape

(2019, 2)

In [52]:
df.info

<bound method DataFrame.info of                                               first_seg  \
0     Sooo Manti Te'o was having a online/phone rela...   
1     this lil girl aint going to win im the king of...   
2     He up stairs rights now and I'm down here gett...   
3     Shit I Am Who Am..Fresh up out of Apologize..I...   
4     It's very rare that I get what I want. Now tha...   
...                                                 ...   
2014  At least I'm happy with knowing I'm always coo...   
2015  I love when you ad lip. If that's what you cal...   
2016  ISO a nice place to do karaoke or DJ music tha...   
2017  Must be an establishment with a "Grown Folks" ...   
2018  , so why i went to the store to get somethingg...   

                                         second_segs  
0                                 to be a female -_-  
1                          she better fall back ....  
2                               to go down nite nite  
3                            OR MOVE TH

## Tokenizer and Model

In [8]:
# PRE_TRAINED_MODEL_NAME = "nlptown/bert-base-multilingual-uncased-sentiment"
# tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
# model = AutoModelForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME)
# classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=0)
classifier = pipeline('sentiment-analysis', device=0)

## Choosing Sequence Length

In [9]:
token_lens = []
for txt in df.txt:
    tokens = tokenizer.encode(txt, max_length=32)
    token_lens.append(len(tokens))

sns.distplot(token_lens)
plt.xlim([0, 32]);
plt.xlabel('Token count');

NameError: name 'tokenizer' is not defined

## Run

In [ ]:
with open('runs/02 EMNLP SAE-AAVE Pairs/aave_gen_txt_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
    writer = csv.writer(f)
    for i, txt in enumerate(tq.tqdm(df.txt)):
        results = classifier(txt)
        for result in results:
            writer.writerow([i, result['label'], round(result['score'], 4)])

  0%|          | 0/2019 [00:00<?, ?it/s]

## Stats

In [66]:
aave_first_pd = pd.read_csv("runs/02 EMNLP SAE-AAVE Pairs/aave_first_seg_sentimental_analysis.csv", header=None)
aave_second_pd = pd.read_csv("runs/02 EMNLP SAE-AAVE Pairs/aave_second_seg_sentimental_analysis.csv", header=None)
aave_gen_pd = pd.read_csv("runs/02 EMNLP SAE-AAVE Pairs/aave_gen_txt_sentimental_analysis.csv", header=None)
sae_first_pd = pd.read_csv("runs/02 EMNLP SAE-AAVE Pairs/sae_first_seg_sentimental_analysis.csv", header=None)
sae_second_pd = pd.read_csv("runs/02 EMNLP SAE-AAVE Pairs/sae_second_seg_sentimental_analysis.csv", header=None)
sae_gen_pd = pd.read_csv("runs/02 EMNLP SAE-AAVE Pairs/sae_gen_txt_sentimental_analysis.csv", header=None)

In [71]:
aave_first_pd.iloc[:, 1].value_counts()

NEGATIVE    1429
POSITIVE     590
Name: 1, dtype: int64

In [72]:
aave_second_pd.iloc[:, 1].value_counts()

NEGATIVE    1166
POSITIVE     853
Name: 1, dtype: int64

In [73]:
aave_gen_pd.iloc[:, 1].value_counts()

NEGATIVE    1161
POSITIVE     858
Name: 1, dtype: int64

In [74]:
sae_first_pd.iloc[:, 1].value_counts()

NEGATIVE    1251
POSITIVE     768
Name: 1, dtype: int64

In [75]:
sae_second_pd.iloc[:, 1].value_counts()

POSITIVE    1019
NEGATIVE    1000
Name: 1, dtype: int64

In [76]:
sae_gen_pd.iloc[:, 1].value_counts()

NEGATIVE    1185
POSITIVE     834
Name: 1, dtype: int64